# INSTALLATIONS

In [1]:
# Core ML and Data Processing
!pip install torch
!pip install transformers
!pip install datasets
!pip install pandas
!pip install numpy
!pip install scikit-learn

# NLTK for data augmentation
!pip install nltk

# Progress bars and utilities
!pip install tqdm

# HuggingFace Hub for model access
!pip install huggingface_hub

# Additional libraries for data processing and evaluation
!pip install nltk
!pip install -U datasets

# Required librarieS

In [ ]:
import torch
from torch.utils.data import DataLoader

from tqdm import tqdm

from datasets import Dataset
from typing import Dict, Optional, List
import logging

from torch.optim import AdamW

from transformers import get_scheduler

import nltk
from nltk.corpus import wordnet
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
from transformers import MarianMTModel, MarianTokenizer, T5ForConditionalGeneration, T5Tokenizer
import random
import logging
from tqdm import tqdm

# File to import model and dataset
from HateXplain_dataset import HateXplainDataset
from HateXplain_model import HateXplainModel
from utils_ import f1, acc, custom_collate_fn, train_cycle, remove_group

# Load HateXplain

In [18]:
"""load the dataset"""
# Create an instance of the class
hatexplain = HateXplainDataset()

# Load the dataset (this will also process the labels)
HateXplain_dataset = hatexplain.load_dataset()

Attempting to load HateXplain dataset...
Processed 15383 examples from train split
Processed 1922 examples from validation split
Processed 1924 examples from test split
Dataset loaded successfully
Available splits: ['train', 'validation', 'test']


In [19]:
# Print statistics about the dataset
hatexplain.dataset = HateXplain_dataset
hatexplain._processed_dataset = HateXplain_dataset
hatexplain.print_dataset_statistics()


Dataset Statistics:

TRAIN Split Statistics:
------------------------------
Total examples: 15383

Label Distribution:
Hate Speech/Offensive: 9132 (59.36%)
Normal: 6251 (40.64%)

Target Group Distribution:
None: 5509 (35.81%)
African: 2335 (15.18%)
Homosexual: 1161 (7.55%)
Islam: 1156 (7.51%)
Jewish: 1011 (6.57%)
Women: 898 (5.84%)
Other: 883 (5.74%)
Arab: 603 (3.92%)
Refugee: 529 (3.44%)
Caucasian: 386 (2.51%)
Asian: 269 (1.75%)
Men: 236 (1.53%)
Hispanic: 187 (1.22%)
Disability: 53 (0.34%)
Christian: 52 (0.34%)
Heterosexual: 31 (0.20%)
Minority: 24 (0.16%)
Economic: 23 (0.15%)
Indian: 15 (0.10%)
Hindu: 11 (0.07%)
none: 7 (0.05%)
Indigenous: 2 (0.01%)
Bisexual: 1 (0.01%)
Buddhism: 1 (0.01%)

Text Length Statistics:
Average length: 23.47 words
Maximum length: 165 words
Minimum length: 2 words

Hate Speech Distribution by Target Group:
African: 2125 hate speech examples (91.01% of this target)
None: 1099 hate speech examples (19.95% of this target)
Jewish: 939 hate speech examples (92.8

# Load Pre-trained model bert-base-uncased-hatexplain

In [20]:
model_HateXplain_BERT = HateXplainModel(
    model_name = "Hate-speech-CNERG/bert-base-uncased-hatexplain",
    num_labels=3,
    device=None,
    max_length=512,
    f1=f1,
    acc=acc
)

Using device: cpu


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
tokenizer_BERT = model_HateXplain_BERT.tokenizer
model_BERT = model_HateXplain_BERT.model

### Tokenize HateXplain dataset

In [22]:
'''
 apply the function to all the elements in the dataset (individually or in batches)
 https://huggingface.co/docs/datasets/v1.11.0/package_reference/main_classes.html?highlight=dataset%20map#datasets.Dataset.map
 batch mode is very powerful. It allows you to speed up processing
 more info here: https://huggingface.co/docs/datasets/en/about_map_batch
'''
cache_files = {
    "test": ".cache/datasets/hateXplain/hateXplain_test_tokenized.arrow",
    "train": ".cache/datasets/hateXplain/hateXplain_train_tokenized.arrow"
}

tokenized_hatexplain = {
    split: HateXplain_dataset[split].map(
        lambda x: model_HateXplain_BERT.tokenize_function(x, "text"),
        batched=True
    )
    for split in ["train", "test"]
}

print(tokenized_hatexplain)

Map:   0%|          | 0/15383 [00:00<?, ? examples/s]

Map:   0%|          | 0/1924 [00:00<?, ? examples/s]

{'train': Dataset({
    features: ['text', 'labels', 'target_group', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 15383
}), 'test': Dataset({
    features: ['text', 'labels', 'target_group', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1924
})}


In [24]:
#hatexplain.dataset = tokenized_hatexplain
tokenized_processed_HateXplain = remove_group(tokenized_hatexplain)
#tokenized_processed_HateXplain = tokenized_hatexplain
# Check one label under the threshold if its good removed
for group in tokenized_processed_HateXplain["test"]["target_group"]:
    if group == 'disability':
        print("dataset not good processed")
        break

Removing target groups with fewer than {'train': 100, 'test': 20} occurrences for training data...
Processing train split...
Removed 220 examples from train split.
Remaining examples in train split: 15163
Processing test split...
Removed 25 examples from test split.
Remaining examples in test split: 1899


In [25]:
#to test rapidly the code, we can use a smaller dataset
small_train_dataset = tokenized_processed_HateXplain["train"].shuffle(seed=42).select(range(50))
small_eval_dataset = tokenized_processed_HateXplain["test"].shuffle(seed=42).select(range(50))

# all the dataset
train_dataset = tokenized_processed_HateXplain["train"]
eval_dataset = tokenized_processed_HateXplain["test"]

In [26]:
# create a DataLoader for your training and test datasets so you can iterate over batches of data:
train_dataloader = DataLoader(
    train_dataset, #change to train_dataset for full dataset
    batch_size=16,
    shuffle=True,
    collate_fn=custom_collate_fn
)

test_dataloader = DataLoader(
    eval_dataset, #change to eval_dataset for full dataset
    batch_size=16,
    shuffle=False,
    collate_fn=custom_collate_fn
)

In [27]:
# Made on a smaller dataset to test the code rapidly
test_small_dataloader = DataLoader(
    small_eval_dataset, #change to eval_dataset for full dataset
    batch_size=16,
    shuffle=False,
    collate_fn=custom_collate_fn
)

### Initialize Optimizer and Scheduler

In [28]:
#optimizer

optimizer = AdamW(model_BERT.parameters(), lr=5e-5)

#Scheduler
num_epochs = 3
num_training_steps = len(train_dataloader)
# feel free to experiment with different num_warmup_steps
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=1, num_training_steps=num_training_steps
)

metrics = {'ACC': acc, 'F1-weighted': f1}

In [29]:
# test on a small dataset first
_, test_loss, test_metrics = model_HateXplain_BERT.evaluate_BERT_with_bias(test_small_dataloader)

eval Loss: 0.5763,  ACC: 0.8281, F1-weighted: 0.7811


NameError: name 'metrics' is not defined

In [ ]:
# test on full dataset
_, test_loss, test_metrics = model_HateXplain_BERT.evaluate_BERT_with_bias(test_dataloader)

eval Loss: 0.7227,  ACC: 0.6737, F1-weighted: 0.6401
Overall Accuracy: 0.6740
0.6669227667906433
Accuracy for target group 'None': 0.7210
F1 Score for target group 'None': 0.5673
Confusion Matrix for 'None':
[[ 20   3   8]
 [ 37 390  97]
 [ 10  28  63]]

Accuracy for target group 'Refugee': 0.5571
F1 Score for target group 'Refugee': 0.5438
Confusion Matrix for 'Refugee':
[[ 4  1  5]
 [ 0 18 13]
 [ 2 10 17]]

Accuracy for target group 'Other': 0.7481
F1 Score for target group 'Other': 0.7445
Confusion Matrix for 'Other':
[[23  3  2]
 [ 0 25  9]
 [ 7 13 53]]

Accuracy for target group 'Arab': 0.6761
F1 Score for target group 'Arab': 0.6009
Confusion Matrix for 'Arab':
[[33  2 11]
 [ 0  5  3]
 [ 2  5 10]]

Accuracy for target group 'African': 0.6990
F1 Score for target group 'African': 0.5395
Confusion Matrix for 'African':
[[168   9  21]
 [ 10  11  13]
 [ 21  16  30]]

Accuracy for target group 'Women': 0.5909
F1 Score for target group 'Women': 0.5917
Confusion Matrix for 'Women':
[[18 

# Data augmentation

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class DataAugmentor:
    def __init__(
        self,
        device: Optional[str] = None,
        use_back_translation: bool = False,
        use_synonym_replacement: bool = False,
        use_random_deletion: bool = False,
        use_paraphrasing: bool = False
    ):
        """
        Initialize the DataAugmentor with various augmentation techniques.

        Args:
            device: Device to run models on ('cuda' or 'cpu')
            use_back_translation: Whether to use back-translation
            use_synonym_replacement: Whether to use synonym replacement
            use_paraphrasing: Whether to use paraphrasing
        """
        self.device = device if device else ('cuda' if torch.cuda.is_available() else 'cpu')

        # Download required NLTK data
        try:
            nltk.data.find('corpora/wordnet')
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('wordnet')
            nltk.download('punkt')

        # Initialize back-translation models if enabled
        if use_back_translation:
            try:
                logger.info("Initializing translation models...")
                self.en_fr_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-fr').to(self.device)
                self.fr_en_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-fr-en').to(self.device)
                self.en_fr_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
                self.fr_en_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-fr-en')
                logger.info("Translation models initialized successfully")
            except Exception as e:
                logger.error(f"Error initializing translation models: {str(e)}")
                use_back_translation = False

        # Initialize paraphrasing model if enabled
        if use_paraphrasing:
            try:
                logger.info("Initializing T5 model...")
                self.t5_model = T5ForConditionalGeneration.from_pretrained('t5-small').to(self.device)
                self.t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
                logger.info("T5 model initialized successfully")
            except Exception as e:
                logger.error(f"Error initializing T5 model: {str(e)}")
                use_paraphrasing = False

        self.use_back_translation = use_back_translation
        self.use_synonym_replacement = use_synonym_replacement
        self.use_random_deletion = use_random_deletion
        self.use_paraphrasing = use_paraphrasing

    def get_word_sense(self, word: str, context: str):
        """
        Get the most appropriate sense of a word given its context using Lesk algorithm.

        Args:
            word: The word to disambiguate
            context: The context in which the word appears

        Returns:
            The most appropriate synset for the word in the given context
        """
        try:
            # Tokenize the context
            context_tokens = word_tokenize(context.lower())

            # Get the most appropriate sense using Lesk algorithm
            sense = lesk(context_tokens, word)

            if sense is None:
                # If Lesk fails, try to get the first synset
                synsets = wordnet.synsets(word)
                if synsets:
                    return synsets[0]
                return None

            return sense
        except Exception as e:
            logger.error(f"Error in word sense disambiguation: {str(e)}")
            return None

    def get_synonyms_with_sense(self, word: str, context: str) -> List[str]:
        """
        Get synonyms for a word using WordNet, considering the word's sense in context.

        Args:
            word: The word to get synonyms for
            context: The context in which the word appears

        Returns:
            List of synonyms that match the word's sense in context
        """
        synonyms = []

        # Get the appropriate sense of the word
        sense = self.get_word_sense(word, context)

        if sense:
            # Get synonyms from the specific sense
            for lemma in sense.lemmas():
                synonym = lemma.name().replace('_', ' ')
                if synonym != word and synonym not in synonyms:
                    synonyms.append(synonym)
        else:
            # Fallback to getting synonyms from all senses
            for syn in wordnet.synsets(word):
                for lemma in syn.lemmas():
                    synonym = lemma.name().replace('_', ' ')
                    if synonym != word and synonym not in synonyms:
                        synonyms.append(synonym)

        return synonyms

    def synonym_replacement_with_wsd(self, text: str, n: int = 1) -> str:
        """
        Replace n random words with their synonyms, considering word sense in context.

        Args:
            text: The input text
            n: Number of words to replace

        Returns:
            Text with replaced synonyms
        """
        words = text.split()
        n = min(n, len(words))
        new_words = words.copy()

        # Get words that are longer than 3 characters and have synonyms
        random_word_list = []
        for word in words:
            if len(word) > 3:
                synonyms = self.get_synonyms_with_sense(word, text)
                if synonyms:
                    random_word_list.append(word)

        random.shuffle(random_word_list)

        num_replaced = 0
        for random_word in random_word_list:
            synonyms = self.get_synonyms_with_sense(random_word, text)
            if len(synonyms) > 0:
                synonym = random.choice(synonyms)
                new_words = [synonym if word == random_word else word for word in new_words]
                num_replaced += 1
            if num_replaced >= n:
                break

        return ' '.join(new_words)

    def get_synonyms(self, word: str) -> List[str]:
        """Get synonyms for a word using WordNet."""
        synonyms = []
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonym = lemma.name().replace('_', ' ')
                if synonym != word and synonym not in synonyms:
                    synonyms.append(synonym)
        return synonyms

    def synonym_replacement(self, text: str, n: int = 1) -> str:
        """Replace n random words with their synonyms."""
        words = text.split()
        n = min(n, len(words))
        new_words = words.copy()

        random_word_list = list(set([word for word in words if len(word) > 3]))
        random.shuffle(random_word_list)

        num_replaced = 0
        for random_word in random_word_list:
            synonyms = self.get_synonyms(random_word)
            if len(synonyms) > 0:
                synonym = random.choice(synonyms)
                new_words = [synonym if word == random_word else word for word in new_words]
                num_replaced += 1
            if num_replaced >= n:
                break

        return ' '.join(new_words)

    def back_translate(self, text: str) -> str:
        """Translate text to French and back to English."""
        try:
            # English to French
            inputs = self.en_fr_tokenizer(text, return_tensors="pt", padding=True).to(self.device)
            translated = self.en_fr_model.generate(**inputs)
            french = self.en_fr_tokenizer.decode(translated[0], skip_special_tokens=True)

            # French to English
            inputs = self.fr_en_tokenizer(french, return_tensors="pt", padding=True).to(self.device)
            translated = self.fr_en_model.generate(**inputs)
            english = self.fr_en_tokenizer.decode(translated[0], skip_special_tokens=True)

            return english
        except Exception as e:
            logger.error(f"Error in back translation: {str(e)}")
            return text

    def paraphrase(self, text: str) -> str:
        """Generate a paraphrase using T5."""
        try:
            input_text = f"paraphrase: {text}"
            inputs = self.t5_tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(self.device)
            outputs = self.t5_model.generate(
                inputs,
                max_length=512,
                num_beams=4,
                no_repeat_ngram_size=2,
                early_stopping=True
            )
            return self.t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
        except Exception as e:
            logger.error(f"Error in paraphrasing: {str(e)}")
            return text

    def random_deletion(self, text: str, p: float = 0.1) -> str:
        """
        Randomly delete words from the text with probability p.

        Args:
            text: The input text
            p: Probability of deleting each word

        Returns:
            Text with randomly deleted words
        """
        try:
            # Tokenize the text
            words = text.split()

            # Keep words with probability 1-p
            new_words = [word for word in words if random.random() > p]

            # If all words were deleted, keep at least one word
            if len(new_words) == 0:
                new_words = [random.choice(words)]

            return ' '.join(new_words)
        except Exception as e:
            logger.error(f"Error in random deletion: {str(e)}")
        return text

    def augment_text(self, text: str) -> List[str]:
        """Apply all enabled augmentation techniques to the text."""
        augmented_texts = []

        if self.use_back_translation:
            augmented_texts.append(self.back_translate(text))

        if self.use_synonym_replacement:
            # Use both regular synonym replacement
            augmented_texts.append(self.synonym_replacement(text, n=2))

        if self.use_random_deletion:
            # Use random deletion
            augmented_texts.append(self.random_deletion(text, p=0.1))

        if self.use_paraphrasing:
            augmented_texts.append(self.paraphrase(text))

        return augmented_texts

    def augment_dataset(
        self,
        dataset: Dict[str, List],
        target_groups,
        texts_to_augment,
        indices_to_augment
    ) -> Dict[str, List]:
        """
        Augment the dataset for specified target groups.

        Args:
            dataset: Dictionary containing 'train' and 'test' datasets
            target_groups: List of target groups to augment
            augmentation_factor: Number of augmented versions to create per example

        Returns:
            Dictionary containing augmented datasets
        """
        augmented_data = {
            'train': {
                'text': [],
                'labels': [],
                'target_group': []
            },
        }

        target_groups_name = list(target_groups.keys())
        # Process each split
        logger.info(f"\nProcessing {'train'} split...")

        """Already pre-trained
        # First, add all original examples
        logger.info("Adding original examples...")
        for i in tqdm(range(len(dataset[split]['text'])), desc="Original examples"):
            text = dataset[split]['text'][i]
            label = dataset[split]['labels'][i]
            target_group = dataset[split]['target_group'][i]

            augmented_data[split]['text'].append(text)
            augmented_data[split]['labels'].append(label)
            augmented_data[split]['target_group'].append(target_group)
        """


        # Process augmentation in batches
        batch_size = 8
        for i in tqdm(range(0, len(texts_to_augment), batch_size), desc="Augmentation batches"):
            batch_texts = texts_to_augment[i:i + batch_size]
            batch_indices = indices_to_augment[i:i + batch_size]

            for text, idx in zip(batch_texts, batch_indices):
                augmentation_factor = target_groups[dataset['train']['target_group'][idx][0]]
                for _ in range(augmentation_factor):
                    augmented_texts = self.augment_text(text)
                    for aug_text in augmented_texts:
                        augmented_data['train']['text'].append(aug_text)
                        augmented_data['train']['labels'].append(dataset['train']['labels'][idx])
                        augmented_data['train']['target_group'].append(dataset['train']['target_group'][idx])

        # Print statistics
        original_count = len(dataset['train']['text'])
        print(f"Original examples: {original_count}")
        print(f"Total examples: {len(augmented_data['train']['text'])}")

        # Print target group distribution
        if target_groups_name:
            print("\nTarget group distribution:")
            for group in target_groups_name:
                count = augmented_data['train']['target_group'].count(group)
                print(f"{group}: {count} examples")

        return augmented_data

In [26]:
target_groups_to_augment = {
    'Men': 2, # low F1 score
    'Refugee': 2, # medium F1 score
    'Caucasian': 2 # low F1 score
}
target_group_name = list(target_groups_to_augment.keys())
print(target_group_name)

['Men', 'Refugee', 'Caucasian']


In [ ]:
# Remove small groups for data augmentation
HateXplain_processed_dataset = hatexplain.remove_small_groups(HateXplain_dataset)
# Then, augment examples for target groups
logger.info("Generating augmented examples...")
texts_to_augment = []
indices_to_augment = []
# Collect texts that need augmentation
for i in range(len(HateXplain_processed_dataset['train']['text'])):
    if len(HateXplain_processed_dataset['train']['target_group'][i]) == 0:
        continue
    if HateXplain_processed_dataset['train']['target_group'][i][0] in target_group_name:
        texts_to_augment.append(HateXplain_processed_dataset['train']['text'][i])
        indices_to_augment.append(i)
print(len(indices_to_augment))

In [ ]:
# Create new datasets with augmented data

# 1. Back-translation only

model_HateXplain_back_trans = HateXplainModel(
    model_name = "Hate-speech-CNERG/bert-base-uncased-hatexplain",
    num_labels=3,
    device=None,
    max_length=512,
    f1=f1,
    acc=acc
)

model_BERT_back_trans = model_HateXplain_back_trans.model

logger.info("\nGenerating back-translation dataset...")
back_trans_augmentor = DataAugmentor(
    use_back_translation=True,
    use_synonym_replacement=False,
    use_random_deletion=False,
    use_paraphrasing=False
    )

'''back_trans_dataset = back_trans_augmentor.augment_dataset(
    dataset=HateXplain_dataset,
    target_groups=target_groups_to_augment,
)'''
back_trans_dataset = back_trans_augmentor.augment_dataset(
    dataset=HateXplain_processed_dataset,
    target_groups=target_groups_to_augment,
    texts_to_augment=texts_to_augment,
    indices_to_augment=indices_to_augment
)
augmented_back_trans = Dataset.from_dict(back_trans_dataset['train'])
tokenized_augmented_back_trans_train = augmented_back_trans.map(
    lambda x: model_HateXplain_back_trans.tokenize_function(x, "text"),
    batched=True
)
dataloader_augmented_back_trans_train = DataLoader(
    tokenized_augmented_back_trans_train, #change to train_dataset for full dataset
    batch_size=16,
    shuffle=True,
    collate_fn=custom_collate_fn
)

In [ ]:
print(back_trans_dataset)

In [ ]:
# 2. Synonym replacement only

model_HateXplain_synonym = HateXplainModel(
    model_name = "Hate-speech-CNERG/bert-base-uncased-hatexplain",
    num_labels=3,
    device=None,
    max_length=512,
    f1=f1,
    acc=acc
)

model_BERT_synonym = model_HateXplain_synonym.model

logger.info("\nGenerating synonym replacement dataset...")
synonym_augmentor = DataAugmentor(
    use_back_translation=False,
    use_synonym_replacement=True,
    use_random_deletion=False,
    use_paraphrasing=False
    )

synonym_dataset = synonym_augmentor.augment_dataset(
    dataset=HateXplain_processed_dataset,
    target_groups=target_groups_to_augment,
    texts_to_augment=texts_to_augment,
    indices_to_augment=indices_to_augment
)
augmented_synonym = Dataset.from_dict(synonym_dataset['train'])
tokenized_augmented_synonym_train = augmented_synonym.map(
    lambda x: model_HateXplain_synonym.tokenize_function(x, "text"),
    batched=True
)
dataloader_augmented_synonym_train = DataLoader(
    tokenized_augmented_synonym_train, #change to train_dataset for full dataset
    batch_size=16,
    shuffle=True,
    collate_fn=custom_collate_fn
)

Using device: cuda


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Augmentation batches: 100%|██████████| 144/144 [03:45<00:00,  1.57s/it]


Original examples: 15163
Augmented examples: -12861
Total examples: 2302

Target group distribution:
Men: 0 examples
Refugee: 0 examples
Caucasian: 0 examples


Map:   0%|          | 0/2302 [00:00<?, ? examples/s]

In [ ]:
# 3. random deletion only

model_HateXplain_rand_del = HateXplainModel(
    model_name = "Hate-speech-CNERG/bert-base-uncased-hatexplain",
    num_labels=3,
    device=None,
    max_length=512,
    f1=f1,
    acc=acc
)

model_BERT_rand_del = model_HateXplain_rand_del.model

logger.info("\nGenerating WSD-based synonym replacement dataset...")
deletion_augmentor = DataAugmentor(
    use_back_translation=False,
    use_synonym_replacement=False,
    use_random_deletion=True,
    use_paraphrasing=False)

deletion_dataset = deletion_augmentor.augment_dataset(
    dataset=HateXplain_processed_dataset,
    target_groups=target_groups_to_augment,
    texts_to_augment=texts_to_augment,
    indices_to_augment=indices_to_augment
)
augmented_deletion = Dataset.from_dict(deletion_dataset['train'])
tokenized_augmented_deletion_train = augmented_deletion.map(
    lambda x: model_HateXplain_rand_del.tokenize_function(x, "text"),
    batched=True
)
dataloader_augmented_deletion_train = DataLoader(
    tokenized_augmented_deletion_train, #change to train_dataset for full dataset
    batch_size=16,
    shuffle=True,
    collate_fn=custom_collate_fn
)


Using device: cuda


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Augmentation batches: 100%|██████████| 144/144 [03:40<00:00,  1.53s/it]


Original examples: 15163
Augmented examples: -12861
Total examples: 2302

Target group distribution:
Men: 0 examples
Refugee: 0 examples
Caucasian: 0 examples


Map:   0%|          | 0/2302 [00:00<?, ? examples/s]

In [ ]:
# 4. Paraphrasing only

model_HateXplain_paraphrase = HateXplainModel(
    model_name = "Hate-speech-CNERG/bert-base-uncased-hatexplain",
    num_labels=3,
    device=None,
    max_length=512,
    f1=f1,
    acc=acc
)

model_BERT_paraphrase = model_HateXplain_paraphrase.model

logger.info("\nGenerating paraphrasing dataset...")
paraphrase_augmentor = DataAugmentor(
    use_back_translation=False,
    use_synonym_replacement=False,
    use_random_deletion=False,
    use_paraphrasing=True)

paraphrase_dataset = paraphrase_augmentor.augment_dataset(
    dataset=HateXplain_processed_dataset,
    target_groups=target_groups_to_augment,
    texts_to_augment=texts_to_augment,
    indices_to_augment=indices_to_augment
)
augmented_paraphrase = Dataset.from_dict(paraphrase_dataset['train'])
tokenized_augmented_paraphrase_train = augmented_paraphrase.map(
    lambda x: model_HateXplain_paraphrase.tokenize_function(x, "text"),
    batched=True
)
dataloader_augmented_paraphrase_train = DataLoader(
    tokenized_augmented_paraphrase_train, #change to train_dataset for full dataset
    batch_size=16,
    shuffle=True,
    collate_fn=custom_collate_fn
)

Using device: cuda


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Augmentation batches: 100%|██████████| 144/144 [20:09<00:00,  8.40s/it]


Original examples: 15163
Augmented examples: -12861
Total examples: 2302

Target group distribution:
Men: 0 examples
Refugee: 0 examples
Caucasian: 0 examples


Map:   0%|          | 0/2302 [00:00<?, ? examples/s]

In [ ]:
# 5. All techniques combined

model_HateXplain_combined = HateXplainModel(
    model_name = "Hate-speech-CNERG/bert-base-uncased-hatexplain",
    num_labels=3,
    device=None,
    max_length=512,
    f1=f1,
    acc=acc
)

model_BERT_combined = model_HateXplain_combined.model

logger.info("\nGenerating combined augmentation dataset...")
combined_augmentor = DataAugmentor(
    use_back_translation=True,
    use_synonym_replacement=True,
    use_random_deletion=True,
    use_paraphrasing=True
)

combined_dataset = combined_augmentor.augment_dataset(
    dataset=HateXplain_dataset,
    target_groups=target_groups_to_augment,
    texts_to_augment=texts_to_augment,
    indices_to_augment=indices_to_augment
)
augmented_combined = Dataset.from_dict(combined_dataset['train'])
tokenized_augmented_combined_train = augmented_combined.map(
    lambda x: model_HateXplain_combined.tokenize_function(x, "text"),
    batched=True
)
dataloader_augmented_combined_train = DataLoader(
    tokenized_augmented_combined_train, #change to train_dataset for full dataset
    batch_size=16,
    shuffle=True,
    collate_fn=custom_collate_fn
)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


839


Augmentation batches:  52%|█████▏    | 55/105 [18:04<14:53, 17.87s/it]

### Train and eval with Back translation

In [ ]:
num_epochs = 3
num_training_steps = len(dataloader_augmented_back_trans_train)

optimizer_back_trans = AdamW(model_BERT_back_trans.parameters(), lr=5e-5)

lr_scheduler_back_trans = get_scheduler(
    name="linear", optimizer=optimizer_back_trans, num_warmup_steps=1, num_training_steps=num_training_steps
)

train_cycle(
      model_HateXplain_back_trans,
      optimizer_back_trans,
      lr_scheduler_back_trans,
      dataloader_augmented_back_trans_train,
      test_dataloader,
      n_epochs=num_epochs,
      num_training_steps=num_training_steps
)

### Train and eval with Synonym

In [ ]:
num_epochs = 3
num_training_steps = len(dataloader_augmented_synonym_train)

optimizer_synonym = AdamW(model_BERT_synonym.parameters(), lr=5e-5)

lr_scheduler_synonym = get_scheduler(
    name="linear", optimizer=optimizer_synonym, num_warmup_steps=1, num_training_steps=num_training_steps
)

train_cycle(
      model_HateXplain_synonym,
      optimizer_synonym,
      lr_scheduler_synonym,
      dataloader_augmented_synonym_train,
      test_dataloader,
      n_epochs=num_epochs,
      num_training_steps=num_training_steps
)

Epoch 0 of 2


  0%|          | 0/144 [00:00<?, ?it/s]

Batch type: <class 'dict'>


  1%|          | 1/144 [00:00<01:52,  1.27it/s]

Batch type: <class 'dict'>


...

Batch type: <class 'dict'>


 99%|█████████▉| 143/144 [01:51<00:00,  1.28it/s]

Batch type: <class 'dict'>


100%|██████████| 144/144 [01:52<00:00,  1.28it/s]


train Loss: 0.5184,  ACC: 0.8142, F1-weighted: 0.7849
eval Loss: 0.8008,  ACC: 0.6526, F1-weighted: 0.6187
Overall Accuracy: 0.6524
0.6505956142650885
Accuracy for target group 'None': 0.6280
F1 Score for target group 'None': 0.5141
Confusion Matrix for 'None':
[[ 22   1   8]
 [ 63 314 147]
 [ 10  15  76]]

Accuracy for target group 'Refugee': 0.6286
F1 Score for target group 'Refugee': 0.6427
Confusion Matrix for 'Refugee':
[[ 7  0  3]
 [ 1 21  9]
 [ 2 11 16]]

Accuracy for target group 'Other': 0.7111
F1 Score for target group 'Other': 0.6593
Confusion Matrix for 'Other':
[[23  1  4]
 [ 3 11 20]
 [ 6  5 62]]

Accuracy for target group 'Arab': 0.7324
F1 Score for target group 'Arab': 0.6252
Confusion Matrix for 'Arab':
[[38  1  7]
 [ 2  3  3]
 [ 5  1 11]]

Accuracy for target group 'African': 0.7023
F1 Score for target group 'African': 0.5413
Confusion Matrix for 'African':
[[169  12  17]
 [ 11  10  13]
 [ 26  10  31]]

Accuracy for target group 'Women': 0.6818
F1 Score for target gro

### Train and eval with random deletion

In [ ]:
num_epochs = 3
num_training_steps = len(dataloader_augmented_deletion_train)

optimizer_rand_del = AdamW(model_BERT_rand_del.parameters(), lr=5e-5)

lr_scheduler_rand_del = get_scheduler(
    name="linear", optimizer=optimizer_rand_del, num_warmup_steps=1, num_training_steps=num_training_steps
)

train_cycle(
      model_HateXplain_rand_del,
      optimizer_rand_del,
      lr_scheduler_rand_del,
      dataloader_augmented_deletion_train,
      test_dataloader,
      n_epochs=num_epochs,
      num_training_steps=num_training_steps
)

Epoch 0 of 2


  0%|          | 0/144 [00:00<?, ?it/s]

Batch type: <class 'dict'>


  1%|          | 1/144 [00:01<02:34,  1.08s/it]

Batch type: <class 'dict'>


...

Batch type: <class 'dict'>


 99%|█████████▉| 143/144 [01:51<00:00,  1.28it/s]

Batch type: <class 'dict'>


100%|██████████| 144/144 [01:52<00:00,  1.28it/s]


train Loss: 0.4941,  ACC: 0.8294, F1-weighted: 0.8046
eval Loss: 0.8276,  ACC: 0.6453, F1-weighted: 0.6140
Overall Accuracy: 0.6451
0.6454077500583693
Accuracy for target group 'None': 0.6189
F1 Score for target group 'None': 0.5015
Confusion Matrix for 'None':
[[ 19   1  11]
 [ 53 310 161]
 [ 11  13  77]]

Accuracy for target group 'Refugee': 0.5857
F1 Score for target group 'Refugee': 0.5796
Confusion Matrix for 'Refugee':
[[ 5  0  5]
 [ 1 18 12]
 [ 2  9 18]]

Accuracy for target group 'Other': 0.7259
F1 Score for target group 'Other': 0.6865
Confusion Matrix for 'Other':
[[24  1  3]
 [ 2 13 19]
 [ 7  5 61]]

Accuracy for target group 'Arab': 0.6901
F1 Score for target group 'Arab': 0.6046
Confusion Matrix for 'Arab':
[[34  2 10]
 [ 0  4  4]
 [ 3  3 11]]

Accuracy for target group 'African': 0.6990
F1 Score for target group 'African': 0.5512
Confusion Matrix for 'African':
[[165   6  27]
 [  8  11  15]
 [ 22  12  33]]

Accuracy for target group 'Women': 0.7000
F1 Score for target gro

### Train and eval with paraphrase

In [ ]:
num_epochs = 3
num_training_steps = len(dataloader_augmented_paraphrase_train)

optimizer_paraphrase = AdamW(model_BERT_paraphrase.parameters(), lr=5e-5)

lr_scheduler_paraphrase = get_scheduler(
    name="linear", optimizer=optimizer_paraphrase, num_warmup_steps=1, num_training_steps=num_training_steps
)

train_cycle(
      model_HateXplain_paraphrase,
      optimizer_paraphrase,
      lr_scheduler_paraphrase,
      dataloader_augmented_paraphrase_train,
      test_dataloader,
      n_epochs=num_epochs,
      num_training_steps=num_training_steps
)

Epoch 0 of 2


  0%|          | 0/144 [00:00<?, ?it/s]

Batch type: <class 'dict'>


  1%|          | 1/144 [00:00<01:53,  1.26it/s]

Batch type: <class 'dict'>


...

Batch type: <class 'dict'>


 99%|█████████▉| 143/144 [01:51<00:00,  1.28it/s]

Batch type: <class 'dict'>


100%|██████████| 144/144 [01:52<00:00,  1.28it/s]


train Loss: 0.6233,  ACC: 0.7225, F1-weighted: 0.6807
eval Loss: 0.8668,  ACC: 0.6288, F1-weighted: 0.5944
Overall Accuracy: 0.6288
0.626783890064037
Accuracy for target group 'None': 0.6052
F1 Score for target group 'None': 0.4907
Confusion Matrix for 'None':
[[ 20   2   9]
 [ 52 307 165]
 [ 13  18  70]]

Accuracy for target group 'Refugee': 0.5857
F1 Score for target group 'Refugee': 0.5903
Confusion Matrix for 'Refugee':
[[ 6  0  4]
 [ 0 22  9]
 [ 3 13 13]]

Accuracy for target group 'Other': 0.7111
F1 Score for target group 'Other': 0.6822
Confusion Matrix for 'Other':
[[23  0  5]
 [ 0 14 20]
 [ 9  5 59]]

Accuracy for target group 'Arab': 0.6620
F1 Score for target group 'Arab': 0.5510
Confusion Matrix for 'Arab':
[[35  0 11]
 [ 1  3  4]
 [ 4  4  9]]

Accuracy for target group 'African': 0.7157
F1 Score for target group 'African': 0.5714
Confusion Matrix for 'African':
[[166   7  25]
 [ 12  10  12]
 [ 22   7  38]]

Accuracy for target group 'Women': 0.5909
F1 Score for target grou

### Train and eval with all combined

In [ ]:

num_epochs = 3
num_training_steps = len(dataloader_augmented_combined_train)

optimizer_combined = AdamW(model_BERT_combined.parameters(), lr=5e-5)

lr_scheduler_combined = get_scheduler(
    name="linear", optimizer=optimizer_combined, num_warmup_steps=1, num_training_steps=num_training_steps
)

train_cycle(
      model_HateXplain_combined,
      optimizer_combined,
      lr_scheduler_combined,
      dataloader_augmented_combined_train,
      test_dataloader,
      n_epochs=num_epochs,
      num_training_steps=num_training_steps
)